In [12]:
import numpy as np
import time
import random
import gym

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp

import matplotlib.pyplot as plt

from IPython.display import clear_output
%matplotlib inline

### Checking for cuda device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [21]:
env = gym.make('Pendulum-v0')
print('Observation Shape:', env.observation_space.shape, '\nAction Shape:', env.action_space)

Observation Shape: (17,) 
Action Shape: Box(6,)


In [5]:
LEARNING_RATE = 0.001
DISCOUNT = 0.99
EPS = 1
EPS_DECAY = 0.9999
END_EPS = 0.1

N_EPISODE = 2000


# Dimensions of input and output of environment
obs_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

In [7]:
class ActorCritic(nn.Module):
    def __init__(self, observations, actions):
        super(ActorCritic, self).__init__()
        self.a = nn.Linear(observations, 128)
        self.mean = nn.Linear(128, actions)
        self.variance = nn.Linear(128, actions)

        self.c = nn.Linear(observations, 128)
        self.value = nn.Linear(128, actions)

    def forward(self, x):
        a = F.relu6(self.a(x))
        mean = 2*F.tanh(self.mean(a))
        variance = F.softplus(self.variance(a)) + 0.0001    # to avoid zero value

        c = F.relu6(self.c(x))
        value = self.value(c)

        return mean, variance, value
    
    def actor_action(state):
        mean, variance, _ = self.forward(state)
        
        m = torch.distributions.Normal(mean, torch.sqrt(variance))
        action = m.sample()
        log_prob = m.log_prob(action)
        return action.detach().cpu().numpy(), log_prob
    
    def loss_function(self, log_prob, )

In [ ]:
class Trainer(mp.Process):
    def __init__(self, global_ac, optimizer, global_ep, global_ep_reward, queue, name):
        super(Trainer, self).__init__()
        self.name = 'trainer_'+str(name)
        self.global_ep, self.global_ep_reward, self.queue = global_ep, global_ep_reward, queue
        self.global_ac, self.optimizer = global_ac, optimizer
        
        self.local_ac = ActorCritic(obs_dim, action_dim)
        self.env = gym.make('Pendulum-v0')
        
    def run(self):
        while self.global_ep.value<=N_EPISODE:
            state = env.reset()
            collect_rewards = []
            log_probs = []
            values = []
            episode_reward = 0
            while True:
                state = torch.FloatTensor(state).to(device)
                action, log_prob = self.local_ac.actor_action(state)
                next_state, reward, done, _ = self.env(action)
                
                if done:
                    break
                episode_reward += reward
                collect_rewards.append(reward)
                
        

In [19]:
global_ac = ActorCritic(obs_dim, action_dim)
global_ac.share_memory()
optimizer = optim.Adam(global_ac.parameters(), lr = LEARNING_RATE)
global_ep, global_ep_reward, queue = mp.Value('i', 0), mp.Value('d', 0.), mp.Queue()

trainers = [Trainer(global_ac, optimizer, global_ep, global_ep_reward, queue, i) for i in range(mp.cpu_count())]
[t.start() for t in trainers]

## Obtain 

NameError: name 'Trainer' is not defined